Interoperability with cuCIM (only Linux)

In [1]:

import os
from matplotlib import pyplot as plt
import cupy as cp
from cucim.skimage.filters import gaussian

os.chdir("/mnt/c/Users/smatysik/projects/nvImageCodecsWSL2/build/example/example_decode")
import nvimgcodecs

ModuleNotFoundError: No module named 'cucim'

In [ ]:
nv_img4k = nvimgcodecs.imread("4k_lossless.jp2")

Convert to cupy ndarray

In [ ]:
%%time
cp_img4k = cp.asarray(nv_img4k)
# CHW(c-style) = Planar -> HWC (c-style) = Interleave
print(cp_img4k.__cuda_array_interface__)

cp_img4k = cp.moveaxis(cp_img4k, (2, 1, 0), (1, 0, 2))
# In Fortran the most rapidly changing index comes first
cp_img4k = cp.asfortranarray(cp_img4k)
cp_img4k = cp.ascontiguousarray(cp_img4k)

print(cp_img4k.__cuda_array_interface__)
cp_img_4k_single_channel = cp_img4k[:, :, 1]

In [ ]:
cp_img4k_blurred = gaussian(cp_img_4k_single_channel, sigma=5)
cp_img4k_blurred_color = gaussian(nv_img4k, sigma=25)
image = cp.asnumpy(cp_img4k_blurred)
plt.imshow(image, cmap='gray')

In [ ]:
from cucim.skimage.filters import threshold_otsu
# determine threshold
threshold = threshold_otsu(cp_img4k_blurred)

# binarize image by apply the threshold
cp_binary_gpu = cp_img4k_blurred > threshold

image = cp.asnumpy(cp_binary_gpu)
plt.imshow(image, cmap='gray')


In [ ]:
from cucim.skimage.morphology import binary_erosion, disk

eroded_gpu = binary_erosion(cp_binary_gpu, selem=disk(2))
image = cp.asnumpy(eroded_gpu)
plt.imshow(image, cmap='gray')


In [ ]:
print(cp_img4k_blurred_color.__cuda_array_interface__)

In [ ]:
%%time
cp_img_planar = cp.moveaxis(cp_img4k_blurred_color, (0, 1), (1, 0))
cp_img_planar = cp.asfortranarray(cp_img_planar)
cp_img_planar = cp.moveaxis(cp_img_planar, (0, 1), (1, 0))
nv_img = nvimgcodecs.asimage(cp_img_planar)


In [ ]:
%%time
nvimgcodecs.imwrite("blurred.j2k", nv_img)
